### __Conexion a la Base de Datos__

In [7]:
import mysql.connector
import pandas as pd

#& Establecemos la conexion a la base de datosS

conn = mysql.connector.connect(
    host="localhost",
    user="root",         
    password="Ha1Da40318",
    charset='utf8mb4',
    use_unicode=True
)

conexion = conn.cursor()
print("Conexion Exitosa")


Conexion Exitosa


### __Creacion de la Base de DAtos__

In [9]:

#? Creamos la base de datos si no existe
conexion.execute("CREATE DATABASE IF NOT EXISTS dss_criptomonedas")
conexion.execute("USE dss_criptomonedas")

print("Base de Datos Creada Exitosamente")

Base de Datos Creada Exitosamente


### __Creación de las tablas del Modelo Constelación__

#### _Creación Tabla Dia Semana_

In [3]:
conexion.execute("""
CREATE TABLE IF NOT EXISTS DiaSemana (
    idDiaSemana INT AUTO_INCREMENT PRIMARY KEY,
    NombreDia VARCHAR(15) NOT NULL
);
""")


#### _Creación Tabla Fecha_

In [4]:
conexion.execute("""
CREATE TABLE IF NOT EXISTS Fecha (
    idFecha INT AUTO_INCREMENT PRIMARY KEY,
    idDiaSemana INT,
    Fecha DATE NOT NULL,
    Hora TIME NOT NULL,
    Mes VARCHAR(20) NOT NULL,
    Año INT NOT NULL,
    Semestre INT NOT NULL,
    Trimestre INT NOT NULL,
    FOREIGN KEY (idDiaSemana) REFERENCES DiaSemana(idDiaSemana)
);
""")


#### _Creación Tabla Moneda_

In [5]:
conexion.execute("""
CREATE TABLE IF NOT EXISTS Moneda (
    idMoneda INT AUTO_INCREMENT PRIMARY KEY,
    PrecioInicial DECIMAL(18,4) NOT NULL,
    Nombre VARCHAR(30) NOT NULL UNIQUE,
    Simbolo VARCHAR(5) NOT NULL
);
""")

#### _Creación Tabla Moneda_

In [6]:
conexion.execute("""
CREATE TABLE IF NOT EXISTS Moneda (
    idMoneda INT AUTO_INCREMENT PRIMARY KEY,
    PrecioInicial DECIMAL(18,4) NOT NULL,
    Nombre VARCHAR(30) NOT NULL UNIQUE,
    Simbolo VARCHAR(5) NOT NULL
);
""")

#### _Creación Tabla Transaccion_

In [8]:
conexion.execute("""
CREATE TABLE IF NOT EXISTS Transaccion (
    idTransaccion INT AUTO_INCREMENT PRIMARY KEY,
    idFecha INT,
    idMoneda INT,
    Precio_Apertura DECIMAL(18,4) NOT NULL,
    Precio_Cierre DECIMAL(18,4) NOT NULL,
    Precio_Maximo DECIMAL(18,4) NOT NULL,
    Precio_Minimo DECIMAL(18,4) NOT NULL,
    Volumen DECIMAL(18,4) NOT NULL,
    FOREIGN KEY (idFecha) REFERENCES Fecha(idFecha),
    FOREIGN KEY (idMoneda) REFERENCES Moneda(idMoneda)
);
""")

#### _Creación Tabla Tendencia_

In [7]:
conexion.execute("""
CREATE TABLE IF NOT EXISTS Tendencia (
    idTendencia INT AUTO_INCREMENT PRIMARY KEY,
    idFecha INT,
    idMoneda INT,
    Variación DECIMAL(18,4) NOT NULL,
    PrecioPromedio DECIMAL(18,4) NOT NULL,
    FOREIGN KEY (idFecha) REFERENCES Fecha(idFecha),
    FOREIGN KEY (idMoneda) REFERENCES Moneda(idMoneda)
);
""")

print("Creación de Tablas terminada con éxito")

Creación de Tablas terminada con éxito


### __Carda de Datos desde el Archivo CSV Limpio__

### _Lectura del archivo de datos atomizados_

In [3]:

df = pd.read_csv("DATA/data_limpia/ARCHIVO_ATOMICO/ARCHIVO_ATOMIZADO.csv")


### _Verificamos las columnas del dataset_

In [4]:
print("Columnas", df.columns.tolist())
print(df.head(3))


Columnas ['PRECIO_APERTURA', 'PRECIO_MAXIMO', 'PRECIO_MINIMO', 'PRECIO_CIERRE', 'VOLUMEN', 'MONEDA', 'ANIO', 'MES', 'DIA', 'HORA']
   PRECIO_APERTURA  PRECIO_MAXIMO  PRECIO_MINIMO  PRECIO_CIERRE    VOLUMEN  \
0           324.08         325.99         323.99         325.40  861.02593   
1           325.42         326.32         325.13         325.91  419.92816   
2           325.84         326.27         325.00         325.75  360.32330   

  MONEDA  ANIO  MES  DIA   HORA  
0    BCH  2020    3    3  17:15  
1    BCH  2020    3    3  17:30  
2    BCH  2020    3    3  17:45  


#### _Iniciamos el Proceso de Carga de los Datos a las Tablas_

##### Formateamos las columnas del dataset

In [5]:
from datetime import datetime

#? Convertimosa a mayusculas los registros del nombre de las monedas
df['MONEDA'] = df['MONEDA'].str.upper()

#? Formateamos a dos digitos el valor del mes y dia
df['MES'] = df['MES'].astype(str).str.zfill(2)
df['DIA'] = df['DIA'].astype(str).str.zfill(2)

#? Crearmoms una columna de fecha y obtene,ps nombre del día
df['FECHA'] = pd.to_datetime(df['ANIO'].astype(str) + '-' + df['MES'] + '-' + df['DIA'])
df['NOMBRE_DIA'] = df['FECHA'].dt.day_name()

#? Convertimos la hora a tipo TIME
df['HORA'] = pd.to_datetime(df['HORA'], format="%H:%M").dt.time

print(df['NOMBRE_DIA'].unique())


['Tuesday' 'Wednesday' 'Thursday' 'Friday' 'Saturday' 'Sunday' 'Monday']


##### Insertamos Dias de la Semana Unicos

In [11]:

#? Creamos una columna con nombre del día en inglés
df['NOMBRE_DIA_EN'] = df['FECHA'].dt.day_name()

#? Definimos un Diccionario de traducción
dias_es = {
    'Monday': 'Lunes',
    'Tuesday': 'Martes',
    'Wednesday': 'Miércoles',
    'Thursday': 'Jueves',
    'Friday': 'Viernes',
    'Saturday': 'Sábado',
    'Sunday': 'Domingo'
}

#? Traducimos al español
df['NOMBRE_DIA'] = df['NOMBRE_DIA_EN'].map(dias_es)

#?Ordenamos los dias a insertar
orden_dias = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']

dias_a_insertar = []


#? Obtenemos los días únicos presentes en el DataFrame
dias_unicos = df['NOMBRE_DIA'].unique()

for dia in orden_dias:
    if dia in dias_unicos:
        dias_a_insertar.append(dia)


#? Verificamos y mostramos
print("Días traducidos:", dias_unicos)

#? Insertamos los registros en la tabla
for dia in dias_a_insertar:
    try:
        dia_str = str(dia).strip()[:15]
        print(f"Insertando día: {dia_str}")
        conexion.execute("INSERT IGNORE INTO DiaSemana (NombreDia) VALUES (%s)", (dia_str,))
    except Exception as e:
        print(f"Error insertando '{dia_str}': {e}")
        
conn.commit()


Días traducidos: ['Martes' 'Miércoles' 'Jueves' 'Viernes' 'Sábado' 'Domingo' 'Lunes']
Insertando día: Lunes
Insertando día: Martes
Insertando día: Miércoles
Insertando día: Jueves
Insertando día: Viernes
Insertando día: Sábado
Insertando día: Domingo


##### Insertamos Monedas y valores asociados unicas

In [12]:
monedas = df['MONEDA'].dropna().unique()
for moneda in monedas:
    print(f"Insertando {moneda}")

Insertando BCH
Insertando BITCOIN
Insertando BNB
Insertando ETHEREUM
Insertando SOLANA


In [14]:

#? Diccionario para mapear los identificadores de la cripto con (Nombre completo, Símbolo)
mapa_nombre_simbolo = {
    'BCH': ('Bitcoin Cash', 'BCH'),
    'BITCOIN': ('Bitcoin', 'BTC'),
    'BNB': ('Binance Coin', 'BNB'),
    'ETHEREUM': ('Ethereum', 'ETH'),
    'SOLANA': ('Solana', 'SOL')
}

#? Insertamos monedas únicas (usamos nombre completo y símbolo)
monedas = df['MONEDA'].dropna().unique()
for identificador in monedas:
    #? Obtenemos nombre y símbolo desde el diccionario, o usar el identificador como ambos por defecto
    nombre, simbolo = mapa_nombre_simbolo.get(identificador, (identificador.title(), identificador))
    
    #? Obtenemos el primer precio de apertura como precio inicial
    precio_inicial = df[df['MONEDA'] == identificador]['PRECIO_APERTURA'].iloc[0]

    #? Insertar en la tabla Moneda
    conexion.execute(
        "INSERT IGNORE INTO Moneda (Nombre, Simbolo, PrecioInicial) VALUES (%s, %s, %s)",
        (nombre, simbolo, precio_inicial)
    )
    print(f"Insertando {nombre} ({simbolo}), precio inicial {precio_inicial}")
conn.commit()

Insertando Bitcoin Cash (BCH), precio inicial 324.08
Insertando Bitcoin (BTC), precio inicial 8698.95
Insertando Binance Coin (BNB), precio inicial 19.4093
Insertando Ethereum (ETH), precio inicial 220.08
Insertando Solana (SOL), precio inicial 2.85


##### Insertamos Fechas Unicas

In [ ]:
from datetime import time
#?Realizamos una conexión de escritura
conn_escritura = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Ha1Da40318",
    database="dss_criptomonedas",
    charset='utf8mb4',
    use_unicode=True
)

#? Obtenemos su cursor
cursor_escritura = conn_escritura.cursor()

#? Creamos una conexion secundaria solo para lectura
conn_lectura = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Ha1Da40318",
    database="dss_criptomonedas",
    charset='utf8mb4',
    use_unicode=True
)
cursor_lectura = conn_lectura.cursor(buffered=True)

#? Eliminamos los valores repetidos para obtener fechas unicas de la dimensión
#? que se pueden asociar a los hechos

fechas = df[['FECHA', 'HORA', 'ANIO', 'MES', 'NOMBRE_DIA']].drop_duplicates()

for _, row in fechas.iterrows():
    #? Extraemos la fecha y la convertimos a tipo date
    fecha = row['FECHA']
    if isinstance(fecha, str):
        fecha = datetime.strptime(fecha, '%Y-%m-%d').date()
    
    #? Extraemos la hora directamente
    hora = row['HORA']
    
    #? Calculamos el nombre del mes
    mes_nombre = datetime.strptime(str(fecha.month), "%m").strftime("%B")
    
    #? Extraemos el año
    año = row['ANIO']
    
    #? Calculamos el semestre: 1 si el mes es <= 6, si no 2
    semestre = 1 if fecha.month <= 6 else 2
    
    #? Calculamos el trimestre (1-4) en función del mes
    trimestre = (fecha.month - 1) // 3 + 1

    #? Consultamos el id del día de la semana en la base de datos usando cursor de lectura
    cursor_lectura.execute("SELECT idDiaSemana FROM DiaSemana WHERE NombreDia = %s", (row['NOMBRE_DIA'],))
    resultado = cursor_lectura.fetchone()

    #? Si se encontramos el id del día, lo insertamos en el registro en la tabla Fecha
    if resultado:
        id_dia = resultado[0]

        #*Insertamos la fecha con el cursor de escritura
        cursor_escritura.execute("""
            INSERT IGNORE INTO Fecha (idDiaSemana, Fecha, Hora, Mes, Año, Semestre, Trimestre)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
        """, (id_dia, fecha, hora, mes_nombre, año, semestre, trimestre))
        conn_escritura.commit()
    else:
        print(f"No se encontró idDiaSemana para: {row['NOMBRE_DIA']}")


In [34]:

#! Cerramos cursores y conexiones
cursor_escritura.close()
conn_escritura.close()
cursor_lectura.close()
conn_lectura.close()

#### Insertamos los registros de todas las Transacciones en la BD

In [40]:
from datetime import time

# Cursores para consultas
cursor_fechas = conn.cursor()
cursor_monedas = conn.cursor()
cursor_insert = conn.cursor()

#? Obtener y mapear fechas (Fecha, Hora) → idFecha
cursor_fechas.execute("SELECT idFecha, Fecha, Hora FROM Fecha")
fechas_result = cursor_fechas.fetchall()

def convertir_tiempo(hora):
    if isinstance(hora, time):
        return hora.strftime("%H:%M:%S")
    else:
        return time(hour=hora.seconds // 3600, minute=(hora.seconds // 60) % 60, second=hora.seconds % 60).strftime("%H:%M:%S")

map_fechas = {
    (f.strftime("%Y-%m-%d"), convertir_tiempo(h)): i
    for i, f, h in fechas_result
}

#? Obtener y mapear monedas: MONEDA → idMoneda
cursor_monedas.execute("SELECT idMoneda, Nombre FROM Moneda")
map_monedas = {nombre.upper(): id_ for id_, nombre in cursor_monedas.fetchall()}

#? Insertar transacciones
from datetime import datetime
import pandas as pd

# Cargar y preparar el DataFrame (usa esto solo si no lo has hecho antes)
df = pd.read_csv("DATA/data_limpia/ARCHIVO_ATOMICO/ARCHIVO_ATOMIZADO.csv")
df['MONEDA'] = df['MONEDA'].str.upper()
df['MES'] = df['MES'].astype(str).str.zfill(2)
df['DIA'] = df['DIA'].astype(str).str.zfill(2)
df['FECHA'] = pd.to_datetime(df['ANIO'].astype(str) + '-' + df['MES'] + '-' + df['DIA'])
df['HORA'] = pd.to_datetime(df['HORA'], format="%H:%M").dt.time

# Contador de inserciones
contador = 0

for _, row in df.iterrows():
    key_fecha = (row['FECHA'].strftime("%Y-%m-%d"), row['HORA'].strftime("%H:%M:%S"))
    id_fecha = map_fechas.get(key_fecha)
    id_moneda = map_monedas.get(row['MONEDA'])

    if id_fecha and id_moneda:
        cursor_insert.execute("""
            INSERT INTO Transaccion (
                idFecha, idMoneda,
                Precio_Apertura, Precio_Cierre,
                Precio_Maximo, Precio_Minimo, Volumen
            ) VALUES (%s, %s, %s, %s, %s, %s, %s)
        """, (
            id_fecha, id_moneda,
            row['PRECIO_APERTURA'], row['PRECIO_CIERRE'],
            row['PRECIO_MAXIMO'], row['PRECIO_MINIMO'], row['VOLUMEN']
        ))
        contador += 1

conn.commit()
print(f"Se insertaron {contador} transacciones correctamente.")


Se insertaron 509843 transacciones correctamente.
